https://www.analyticsvidhya.com/blog/2019/04/build-first-multi-label-image-classification-model-python/
https://medium.com/bycodegarage/encoding-categorical-data-in-machine-learning-def03ccfbf40
https://towardsdatascience.com/plant-ai-plant-disease-detection-using-convolutional-neural-network-9b58a96f2289
https://www.pyimagesearch.com/2018/05/07/multi-label-classification-with-keras/


In [122]:
pip install pillow


Note: you may need to restart the kernel to use updated packages.


In [123]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [124]:
import tensorflow
tensorflow.__version__

'2.3.0'

In [125]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [126]:
import keras
keras.__version__

'2.4.3'

In [127]:
import tensorflow as tf  #just to check keras using tensorflow backend
tf.keras.backend.floatx()

'float32'

In [128]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input/plantVillage'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [129]:
import cv2

In [130]:
dir_root="../input/plantdisease/"
def_img_size = tuple((256, 256))


In [131]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, def_img_size)   
            return tf.keras.preprocessing.image.img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [132]:
image_list, label_list = [], []
try:
    print("Loading images ...")
    root_d = listdir(dir_root)  #get the list of all files and directories in the specified directory
    for directory in root_d :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_d.remove(directory)

    for plant_folder in root_d :
        plant_disease_folder_list = listdir(f"{dir_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f" Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{dir_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:250]: #250 images from each folder
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory)) #image into numeric append to image_list
                    label_list.append(plant_disease_folder) #label_list contain corresponding plant disease folder name
    print(" Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

Loading images ...
 Processing Tomato__Tomato_YellowLeaf__Curl_Virus ...
 Processing Pepper__bell___Bacterial_spot ...
 Processing Pepper__bell___healthy ...
 Processing Tomato_healthy ...
 Processing Tomato_Leaf_Mold ...
 Processing Potato___healthy ...
 Processing Tomato_Late_blight ...
 Processing Tomato_Septoria_leaf_spot ...
 Processing Tomato_Spider_mites_Two_spotted_spider_mite ...
 Processing Potato___Late_blight ...
 Processing Tomato_Bacterial_spot ...
 Processing Tomato__Tomato_mosaic_virus ...
 Processing Tomato__Target_Spot ...
 Processing Potato___Early_blight ...
 Processing Tomato_Early_blight ...
 Processing PlantVillage ...
 Image loading completed


In [148]:
print(root_d)
print("\n",plant_disease_folder_list)
print("\n",plant_disease_image_list) 
print(len(image_list)) #list images converted to array numeric data based on pixel values
print(len(label_list)) #list label corresponding to image folder name that is used in labelling 
print(plant_folder)
print(disease_folder)

#print(label_list)

['PlantVillage', 'plantvillage']

 ['PlantVillage']

 ['Tomato__Tomato_YellowLeaf__Curl_Virus', 'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Tomato_healthy', 'Tomato_Leaf_Mold', 'Potato___healthy', 'Tomato_Late_blight', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Potato___Late_blight', 'Tomato_Bacterial_spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Target_Spot', 'Potato___Early_blight', 'Tomato_Early_blight']
3652
3652
plantvillage
PlantVillage


In [149]:
image_size = len(image_list)


Label Binarizer is an SciKit Learn class that accepts Categorical data as input and returns an Numpy array. 

In [150]:
from sklearn import preprocessing
label_binarizer = preprocessing.LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list) #label classes based on label_list
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb')) #saved the label binarizer instance using pickle 
n_classes = len(label_binarizer.classes_)   #classes_: array of shape [n_class]



In [151]:
print(n_classes) #15 classes are there in dataset

15


In [152]:
print(label_binarizer.classes_)  #attribute classes_array of shape [n_class]



['Pepper__bell___Bacterial_spot' 'Pepper__bell___healthy'
 'Potato___Early_blight' 'Potato___Late_blight' 'Potato___healthy'
 'Tomato_Bacterial_spot' 'Tomato_Early_blight' 'Tomato_Late_blight'
 'Tomato_Leaf_Mold' 'Tomato_Septoria_leaf_spot'
 'Tomato_Spider_mites_Two_spotted_spider_mite' 'Tomato__Target_Spot'
 'Tomato__Tomato_YellowLeaf__Curl_Virus' 'Tomato__Tomato_mosaic_virus'
 'Tomato_healthy']


In [140]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0 #scaling data range [0,255] to [0,1] 


255 is max value for R/G/B
trrain_test_split(...)
test_size=0.2 >> 20% test 80%train which is good practise

In [153]:
from sklearn.model_selection import train_test_split
print("Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 0) 


Spliting data to train, test


In [154]:
print(len(x_train))

2921


In [155]:
print(len(x_test))

731


While the word “augment” means to make something “greater” or “increase” something (in this case, data), the Keras ImageDataGenerator class actually works by:

Accepting a batch of images used for training.
Taking this batch and applying a series of random transformations to each image in the batch (including random rotation, resizing, shearing, etc.).
Replacing the original batch with the new, randomly transformed batch.
Training the CNN on this randomly transformed batch (i.e., the original data itself is not used for training).
https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/

In [156]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")
